# Combining main_analysis with garage & basement data

### Pre-process data

#### We have also decided that catboost is the strongest model for our regression problem, so we will go ahead and use it for the combined files

In [112]:
#Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [113]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics

from catboost import CatBoostRegressor, Pool

from sklearn.metrics import mean_squared_error
from math import sqrt

In [114]:
data = r'/Users/OliverPan/Desktop/house/train.csv'

In [115]:
df = pd.read_csv(data)

In [116]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [117]:
new_main_df = df[["Id", "SalePrice", "OverallQual", "LotArea", "GarageFinish", "GarageCars",
       "GarageArea", "BsmtQual", "TotalBsmtSF"]]

In [118]:
new_main_df.head()

,Id,SalePrice,OverallQual,LotArea,GarageFinish,GarageCars,GarageArea,BsmtQual,TotalBsmtSF
0,1,208500,7,8450,RFn,2,548,Gd,856
1,2,181500,6,9600,RFn,2,460,Gd,1262
2,3,223500,7,11250,RFn,2,608,Gd,920
3,4,140000,7,9550,Unf,3,642,TA,756
4,5,250000,8,14260,RFn,3,836,Gd,1145


In [119]:
new_main_df["GarageFinish"] = new_main_df["GarageFinish"].astype(str)
new_main_df["BsmtQual"] = new_main_df["BsmtQual"].astype(str)

/Users/OliverPan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/OliverPan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Analysis

In [120]:
new_main_df.corr()["SalePrice"]

Id            -0.021917
SalePrice      1.000000
OverallQual    0.790982
LotArea        0.263843
GarageCars     0.640409
GarageArea     0.623431
TotalBsmtSF    0.613581
Name: SalePrice, dtype: float64

In [121]:
new_main_df = new_main_df.drop(["Id"], axis = 1)

In [122]:
new_main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   SalePrice     1460 non-null   int64 
 1   OverallQual   1460 non-null   int64 
 2   LotArea       1460 non-null   int64 
 3   GarageFinish  1460 non-null   object
 4   GarageCars    1460 non-null   int64 
 5   GarageArea    1460 non-null   int64 
 6   BsmtQual      1460 non-null   object
 7   TotalBsmtSF   1460 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 91.4+ KB


### CatBoost

In [123]:
X = new_main_df.drop(["SalePrice"], axis = 1)
y = new_main_df[["SalePrice"]]

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [125]:
category = ["OverallQual", "GarageFinish", "GarageCars", "BsmtQual"]

In [126]:
cat_boost = CatBoostRegressor(depth=3, learning_rate=0.1, loss_function='RMSE')
cat_boost.fit(X_train, y_train, cat_features=category)

0:	learn: 74322.1412927	total: 1.58ms	remaining: 1.58s
1:	learn: 70741.5931845	total: 9.31ms	remaining: 4.64s
2:	learn: 67682.0909376	total: 11ms	remaining: 3.66s
3:	learn: 64979.8040554	total: 13.7ms	remaining: 3.41s
4:	learn: 62287.2200930	total: 15.4ms	remaining: 3.07s
5:	learn: 60066.1598539	total: 16.6ms	remaining: 2.75s
6:	learn: 58161.9864038	total: 19.4ms	remaining: 2.75s
7:	learn: 56338.8590969	total: 21.2ms	remaining: 2.63s
8:	learn: 54643.0267186	total: 22.6ms	remaining: 2.49s
9:	learn: 52623.7573529	total: 24ms	remaining: 2.38s
10:	learn: 50898.5480570	total: 25.1ms	remaining: 2.25s
11:	learn: 49208.1000682	total: 26ms	remaining: 2.14s
12:	learn: 47683.8493713	total: 30.7ms	remaining: 2.33s
13:	learn: 46459.2406585	total: 32.1ms	remaining: 2.26s
14:	learn: 45312.9547506	total: 33.1ms	remaining: 2.17s
15:	learn: 44262.1770667	total: 34.6ms	remaining: 2.13s
16:	learn: 43363.7718238	total: 36.1ms	remaining: 2.09s
17:	learn: 42489.9472607	total: 37.2ms	remaining: 2.03s
18:	lear

153:	learn: 29050.2518126	total: 270ms	remaining: 1.49s
154:	learn: 29039.2888593	total: 273ms	remaining: 1.49s
155:	learn: 29024.1483340	total: 274ms	remaining: 1.48s
156:	learn: 29015.9774852	total: 282ms	remaining: 1.51s
157:	learn: 28996.8274010	total: 285ms	remaining: 1.52s
158:	learn: 28984.1297333	total: 286ms	remaining: 1.51s
159:	learn: 28954.5150684	total: 288ms	remaining: 1.51s
160:	learn: 28891.6874057	total: 294ms	remaining: 1.53s
161:	learn: 28860.7872428	total: 296ms	remaining: 1.53s
162:	learn: 28832.2835553	total: 298ms	remaining: 1.53s
163:	learn: 28812.3573174	total: 300ms	remaining: 1.53s
164:	learn: 28770.1165416	total: 301ms	remaining: 1.52s
165:	learn: 28727.9054166	total: 302ms	remaining: 1.51s
166:	learn: 28702.6661697	total: 303ms	remaining: 1.51s
167:	learn: 28673.9578632	total: 304ms	remaining: 1.5s
168:	learn: 28658.4501422	total: 309ms	remaining: 1.52s
169:	learn: 28649.6759591	total: 310ms	remaining: 1.51s
170:	learn: 28612.6093085	total: 311ms	remaining:

335:	learn: 25906.9366600	total: 523ms	remaining: 1.03s
336:	learn: 25880.0329397	total: 525ms	remaining: 1.03s
337:	learn: 25873.7056096	total: 527ms	remaining: 1.03s
338:	learn: 25861.4624340	total: 529ms	remaining: 1.03s
339:	learn: 25838.2792302	total: 535ms	remaining: 1.04s
340:	learn: 25836.4673533	total: 540ms	remaining: 1.04s
341:	learn: 25831.6698537	total: 541ms	remaining: 1.04s
342:	learn: 25827.6995313	total: 542ms	remaining: 1.04s
343:	learn: 25811.2253471	total: 543ms	remaining: 1.03s
344:	learn: 25799.2761644	total: 544ms	remaining: 1.03s
345:	learn: 25794.2061362	total: 545ms	remaining: 1.03s
346:	learn: 25791.4259076	total: 546ms	remaining: 1.03s
347:	learn: 25768.0276470	total: 550ms	remaining: 1.03s
348:	learn: 25761.6289870	total: 551ms	remaining: 1.03s
349:	learn: 25744.9456037	total: 552ms	remaining: 1.02s
350:	learn: 25741.3862880	total: 553ms	remaining: 1.02s
351:	learn: 25732.1819431	total: 555ms	remaining: 1.02s
352:	learn: 25721.6341380	total: 556ms	remaining

519:	learn: 23963.8737390	total: 790ms	remaining: 729ms
520:	learn: 23956.8746055	total: 795ms	remaining: 731ms
521:	learn: 23949.7490694	total: 797ms	remaining: 730ms
522:	learn: 23946.8631897	total: 803ms	remaining: 732ms
523:	learn: 23940.2681993	total: 805ms	remaining: 731ms
524:	learn: 23929.0207690	total: 808ms	remaining: 731ms
525:	learn: 23904.4065713	total: 809ms	remaining: 729ms
526:	learn: 23896.6989954	total: 810ms	remaining: 727ms
527:	learn: 23890.3207312	total: 811ms	remaining: 725ms
528:	learn: 23867.4983700	total: 812ms	remaining: 723ms
529:	learn: 23853.1486139	total: 813ms	remaining: 721ms
530:	learn: 23847.6544914	total: 814ms	remaining: 719ms
531:	learn: 23840.1108336	total: 815ms	remaining: 717ms
532:	learn: 23818.0301485	total: 816ms	remaining: 715ms
533:	learn: 23816.9749756	total: 817ms	remaining: 713ms
534:	learn: 23802.7213966	total: 818ms	remaining: 711ms
535:	learn: 23797.5763976	total: 819ms	remaining: 709ms
536:	learn: 23791.3182419	total: 821ms	remaining

685:	learn: 22498.1613448	total: 1.04s	remaining: 477ms
686:	learn: 22490.0580218	total: 1.04s	remaining: 476ms
687:	learn: 22475.4005758	total: 1.05s	remaining: 477ms
688:	learn: 22472.3562187	total: 1.05s	remaining: 476ms
689:	learn: 22463.9458939	total: 1.05s	remaining: 474ms
690:	learn: 22461.0118281	total: 1.06s	remaining: 473ms
691:	learn: 22456.1364827	total: 1.06s	remaining: 472ms
692:	learn: 22447.5526981	total: 1.06s	remaining: 472ms
693:	learn: 22430.2884297	total: 1.06s	remaining: 470ms
694:	learn: 22418.1326302	total: 1.07s	remaining: 468ms
695:	learn: 22396.2048716	total: 1.07s	remaining: 467ms
696:	learn: 22383.3962875	total: 1.07s	remaining: 465ms
697:	learn: 22371.3558261	total: 1.07s	remaining: 463ms
698:	learn: 22356.3291165	total: 1.07s	remaining: 462ms
699:	learn: 22348.7179703	total: 1.07s	remaining: 460ms
700:	learn: 22341.4251584	total: 1.07s	remaining: 458ms
701:	learn: 22335.3421614	total: 1.07s	remaining: 457ms
702:	learn: 22326.9596064	total: 1.08s	remaining

872:	learn: 21131.1316263	total: 1.31s	remaining: 191ms
873:	learn: 21126.5732715	total: 1.31s	remaining: 190ms
874:	learn: 21118.0841780	total: 1.32s	remaining: 188ms
875:	learn: 21115.4597372	total: 1.33s	remaining: 188ms
876:	learn: 21107.5655568	total: 1.33s	remaining: 187ms
877:	learn: 21106.1643787	total: 1.33s	remaining: 185ms
878:	learn: 21092.1556923	total: 1.34s	remaining: 184ms
879:	learn: 21091.6387673	total: 1.34s	remaining: 182ms
880:	learn: 21091.0127112	total: 1.34s	remaining: 181ms
881:	learn: 21087.1919726	total: 1.34s	remaining: 179ms
882:	learn: 21084.6731980	total: 1.34s	remaining: 178ms
883:	learn: 21080.3042992	total: 1.34s	remaining: 176ms
884:	learn: 21066.5410346	total: 1.34s	remaining: 175ms
885:	learn: 21054.1652917	total: 1.34s	remaining: 173ms
886:	learn: 21049.0546436	total: 1.35s	remaining: 172ms
887:	learn: 21047.4181931	total: 1.35s	remaining: 170ms
888:	learn: 21039.0818817	total: 1.35s	remaining: 169ms
889:	learn: 21033.5441784	total: 1.35s	remaining

In [127]:
y_pred = cat_boost.predict(X_test)

In [128]:
#Results using regressor
df = pd.DataFrame({'Actual': y_test["SalePrice"].to_list(), 'Predicted': list(y_pred.flatten())})
df.head()

,Actual,Predicted
0,200624,244185.527769
1,133000,137357.951645
2,110000,113032.859460
3,192000,209740.247245
4,88000,101901.191340


In [129]:
## We should also look at RMSE
rmse = sqrt(mean_squared_error(y_test, y_pred))
rmse

38214.09107557811

In [130]:
## Normalised RMSE
sqrt(mean_squared_error(y_test, y_pred)) / (y_test["SalePrice"].max() - y_test["SalePrice"].min())

0.0563629661881683

In [131]:
new_main_df.drop(["SalePrice"], axis = 1).columns.values

array(['OverallQual', 'LotArea', 'GarageFinish', 'GarageCars',
       'GarageArea', 'BsmtQual', 'TotalBsmtSF'], dtype=object)

In [132]:
list(cat_boost.get_feature_importance(Pool(X_test, label = y_test, cat_features=category)))

[20.724342544255,
 12.69423212139462,
 2.4332743151805354,
 9.650126357266862,
 18.3006603114762,
 18.56105281487624,
 17.6363115355505]

#### We can see that the model is not predicting the higher Actual value houses very well. There might be additional features that we can include to help with higher valued houses

In [133]:
df[df["Actual"] > 500000]

,Actual,Predicted
8,745000,617553.766727
258,538000,380743.402917
288,555000,470952.969802
